<a href="https://colab.research.google.com/github/nguyen-nhat-mai/methane-leak-detection/blob/main/Web_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Build web app for classifier model

In [58]:
!pip install streamlit -q

Reference:

https://towardsdatascience.com/create-an-image-classification-web-app-using-pytorch-and-streamlit-f043ddf00c24#2b4c
https://www.youtube.com/watch?v=NEhrkeF2o_M

In [69]:
# Write the web app
%%writefile app.py
import streamlit as st
from PIL import Image
from torchvision import models, transforms
import torch
import time
import pandas as pd
import base64
from io import BytesIO

#---------------------- TO UPDATE WITH FINAL MODEL-----------------------#
def predict(image_path):
    # Load model
    best_model = models.resnet101(pretrained=True)
    # Define transformation
    transform = transforms.Compose([
      transforms.Resize(256),
      transforms.CenterCrop(224),
      transforms.ToTensor(),
      transforms.Normalize(
      mean=[0.485, 0.456, 0.406],
      std=[0.229, 0.224, 0.225]
      )])
    # Load data
    img = Image.open(image_path)
    batch_t = torch.unsqueeze(transform(img), 0)
    # Do inference
    best_model.eval()
    out = best_model(batch_t)
    # Load all of the classes => Yes/ No for methane case
    with open('imagenet_classes.txt') as f:
        classes = [line.strip() for line in f.readlines()]
    # Calculate the probability
    prob = torch.nn.functional.softmax(out, dim=1)[0] * 100
    _, indices = torch.sort(out, descending=True)
    # Return the classes and corresponding probability
    return [(classes[idx], prob[idx].item()) for idx in indices[0][:1]]

# ------------------------------WEB APP----------------------------------#
st.set_option('deprecation.showfileUploaderEncoding', False)
st.title("Detect Methane Leaks")
st.write("")



st.subheader("Prediction on uploaded images")
# Define a function to create a download link for a given DataFrame
def download_link(df, filename, text):
    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode('utf-8')).decode()  # base64 encoding
    href = f"data:text/csv;base64,{b64}"
    return f'<a href="{href}" download="{filename}">{text}</a>'

file_up = st.file_uploader("Upload your images", type=["jpg", "jpeg", "png"], accept_multiple_files=True)
if file_up is not None:
    # Print images and their predictions
    all_predictions = pd.DataFrame() # create an empty DataFrame to store all predictions
    for img_file in file_up:
        image = Image.open(img_file)
        st.image(image, caption=img_file.name, use_column_width=True)
        st.write("")
        with st.spinner('Predicting...'):
            predictions_df = predict(img_file)
            # Append the current predictions to the DataFrame along with the file name
            predictions_df = [(img_file.name,)+predictions_df[0]]
            all_predictions = all_predictions.append(predictions_df, ignore_index=True) 
            # Print out the prediction labels with probability
            for i in range(len(predictions_df)):
                st.write("Prediction:", all_predictions.iloc[i][1], "-   Probability (%): ", all_predictions.iloc[i][2])
    
    
    # Display summary and export the predictions to an CSV file
    if not all_predictions.empty:
        st.write("")
        all_predictions.columns = ['File name', 'Label', 'Probability (%)']
        st.subheader("Prediction Detail")
        st.write(all_predictions)
        all_predictions.to_csv("predictions.csv", index=False)
        st.markdown(download_link(all_predictions, "predictions.csv", "Download CSV"), unsafe_allow_html=True)

Overwriting app.py


In [70]:
# Run app.py and made available on a local URL
!streamlit run app.py & npx localtunnel --port 8501

[############......] / install:get-caller-file: info lifecycle get-caller-file@

npx: installed 22 in 4.495s

  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://34.139.171.39:8501

your url is: https://lovely-owls-worry.loca.lt
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/content/app.py:65: FutureWarning: The frame.append method is deprecated and wil